## Recomendador con filtrado colaborativo

Recomendador sin usar el framework de Surprise

### Carga de datos

Es necesario importar las librerías para nuestro recomendador.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import seaborn as sns

Comenzamos cargando los datasets de las canciones junto a los ratings. El archivo original del dataset de las canciones incluía 1.2 millones de canciones por lo que se ha reducido el dataset a 50000 porque al no usar librerías específicas como Surprise no es eficiente y sobrepasa el uso de la memoria si utilizamos el dataset no reducido.

In [2]:
songs = pd.read_csv('spotify_data_red.csv')
#songs = pd.read_csv('spotify_data.csv') Dataset original

In [3]:
ratings = pd.read_csv('rating.csv')

### Explicación de los atributos de las canciones

In [4]:
# cargamos las primeras filas para ver las columnas del dataset
songs.head()

,songId,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


Definición de los atributos
+ ***artist_name***: nombre del artista
- ***track_name***: nombre de la canción
+ ***track_id***: id único de la canción es Spotify
- ***popularity***: cuanto más alto es el valor, más popular es la canción (0-100)
+ ***year***: año de lanzamiento de la canción
- ***genre***: género de la canción
+ ***danceability***: describe lo adecuada que es una pista para bailar basándose en una combinación de elementos musicales como el tempo, la estabilidad del ritmo, la fuerza del compás y la regularidad general. Un valor de 0.0 es el menos bailable y 1.0 el más bailable (0.0-1.0)
- ***energy***: medida que representa una medida perceptiva de intensidad y actividad (0.0-1.0)
+ ***key***: tonalidad de la pista y los números enteros se asignan a tonos utilizando la notación estándar Pitch Class. Por ejemplo, 0 = C, 1 = C♯/D♭, 2 = D, y así sucesivamente. Si no se detectó ninguna clave, el valor es -1 (-1-10)
- ***loudness***: en decibelios (dB). Los valores de sonoridad se promedian en toda la pista y son útiles para comparar la sonoridad relativa de las pistas. La sonoridad es la cualidad de un sonido que es el principal correlato psicológico de la fuerza física (amplitud). Los valores suelen oscilar entre -60 y 0 db (-60-0dB)
+ ***mode***: indica la modalidad (mayor o menor) de una pista, el tipo de escala del que se deriva su contenido melódico. Mayor se representa con 1 y menor con 0
- ***speechiness***: detecta la presencia de palabras habladas en una pista. Cuanto más exclusivamente hablada sea la grabación (por ejemplo, programa de entrevistas, audiolibro, poesía), más se acercará a 1.0 el valor del atributo. Los valores superiores a 0,66 describen pistas que probablemente estén compuestas en su totalidad por palabras habladas. Los valores entre 0.33 y 0.66 describen pistas que pueden contener tanto música como voz, ya sea en secciones o en capas, incluyendo casos como la música rap. Los valores inferiores a 0.33 representan probablemente música y otras pistas no habladas
+ ***acousticness***: medida de confianza de 0.0 a 1.0 de si la pista es acústica. 1,0 representa una confianza alta en que la pista es acústica (0.0-1.0)
- ***instrumentalness***: predice si una pista no contiene voces. Los sonidos "ooh" y "aah" se consideran instrumentales en este contexto. Las pistas de rap o spoken word son claramente "vocales". Cuanto más se acerque el valor de instrumental a 1.0, mayor será la probabilidad de que la pista no contenga voces. Los valores superiores a 0.5 representan pistas instrumentales, pero la confianza es mayor a medida que el valor se acerca a 1.0 (0.0-1.0)
+ ***liveness***: detecta la presencia de público en la grabación. Los valores de liveness más altos representan una mayor probabilidad de que la pista se haya interpretado en directo. Un valor superior a 0.8 indica una gran probabilidad de que la pista se haya grabado en directo (0.0-1.0)
- ***valence***: medida de 0.0 a 1.0 que describe la positividad musical que transmite una pista. Las pistas con valencia alta suenan más positivas (por ejemplo, felices, alegres, eufóricas), mientras que las pistas con valencia baja suenan más negativas (por ejemplo, tristes, deprimidas, enfadadas) (0.0-1.0)
+ ***tempo***: en pulsaciones por minuto (BPM). En terminología musical, el tempo es la velocidad o el ritmo de una pieza determinada y se deriva directamente de la duración media del compás
- ***duration_ms***: duración de la pista en milisegundos
+ ***time_signature***: convención notacional para especificar cuántos tiempos hay en cada compás (3-7)



### Matriz de Correlación entre géneros

Vamos a contar los diferentes géneros que hay para empezar a crear la matriz de correlación entre los distintos géneros.

In [5]:
from collections import Counter
genre_frequency = Counter(songs['genre'])
print(f"Hay {len(genre_frequency)} géneros.")
genre_frequency

Hay 74 géneros.


Counter({'acoustic': 968,
         'afrobeat': 802,
         'alt-rock': 954,
         'ambient': 981,
         'black-metal': 985,
         'blues': 909,
         'breakbeat': 799,
         'cantopop': 934,
         'chicago-house': 244,
         'chill': 944,
         'classical': 805,
         'club': 921,
         'comedy': 979,
         'country': 879,
         'dance': 786,
         'dancehall': 900,
         'death-metal': 885,
         'deep-house': 917,
         'detroit-techno': 119,
         'disco': 756,
         'drum-and-bass': 834,
         'dub': 700,
         'dubstep': 490,
         'edm': 510,
         'electro': 385,
         'electronic': 421,
         'emo': 933,
         'folk': 706,
         'forro': 921,
         'french': 759,
         'funk': 629,
         'garage': 741,
         'german': 788,
         'gospel': 949,
         'goth': 808,
         'grindcore': 854,
         'groove': 713,
         'guitar': 818,
         'hard-rock': 545,
         'hardcore'

Lo escribimos en archivo para una mejor visualización, aunque no es necesario.

In [6]:
# Nombre del archivo
file_name = 'genres.txt'
genres = []

# Abrir el archivo en modo escritura
with open(file_name, 'w') as archivo:
    # Iterar sobre la lista y escribir cada elemento en una nueva línea
    for genre in genre_frequency:
        archivo.write(str(genre) + '\n')
        genres.append(genre)

Obtenemos la similitud entre géneros a través del parecido por texto, es decir, son parecidos los géneros como punk y punk-rock pero entre punk y romance, debido a que la primera pareja contiene punk en ambos.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert genre names to TF-IDF vectors
vectorizer = TfidfVectorizer()
genre_vectors = vectorizer.fit_transform(genres)

# Calculate cosine similarity between genre vectors
cosine_sim_genre = cosine_similarity(genre_vectors, genre_vectors)

In [8]:
cosine_sim_genre #visualizamos la matriz de similitud

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

Para probar el ejemplo comentado arriba, probamos a ver la similitud entre punk y punk-rock y entre punk y romance

In [9]:
print('Similitud entre ', genres[61], ' y ', genres[62], ': ', cosine_sim_genre[61][62])
print('Similitud entre ', genres[61], ' y ', genres[65], ': ', cosine_sim_genre[61][65])

Similitud entre  punk  y  punk-rock :  0.7811868929584836
Similitud entre  punk  y  romance :  0.0


Debido que a nuestro dataset no contiene una lista de todos los géneos de la canción en esta primera fase (antes de la conexión con la API de Spotify) se descarta una posible recomendación por géneros mediante una vectorización de estos en binario. En su lugar usaremos por parecido de nombre con TF-IDF (no se ha aplicado todavía)

### Recomendador Filtrado Colaborativo

Código tomado de los apuntos de la directora del TFG.

El filtrado colaborativo se basa en recomendaciones por personas de gustos similares.

Vamos a crear una matriz de dispersión o de utilidaad (usuario-elemento), para ello hemos creado la función create_X().

In [10]:
from scipy.sparse import csr_matrix

def create_X(df):
    """
    Genera una matriz dispersa a partir del marco de datos de calificaciones.
    
    Argumentos:
        df: dataframe de pandas que contiene 3 columnas (userId, songId, rating)
    
    Devoluciones:
        X: matriz dispersa
        user_mapper: dict que asigna las identificaciones de usuario a los índices de usuario
        user_inv_mapper: dict que asigna índices de usuario a ID de usuario
        song_mapper: dict que asigna las identificaciones de canciones a los índices de canciones
        song_inv_mapper: dict que asigna índices de canciones a ID de canciones
        
    """
    M = df['userId'].nunique()
    N = df['songId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    song_mapper = dict(zip(np.unique(df["songId"]), list(range(N))))
    
    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    song_inv_mapper = dict(zip(list(range(N)), np.unique(df["songId"])))
    
    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [song_mapper[i] for i in df['songId']]

    X = csr_matrix((df["rating"], (user_index,item_index)), shape=(M,N))
    
    return X, user_mapper, song_mapper, user_inv_mapper, song_inv_mapper

X, user_mapper, song_mapper, user_inv_mapper, song_inv_mapper = create_X(ratings)

#### Dispersión

La dispersió se calcula dividiendo el número de elementos almacenados en la matriz entre el número total de elementos.

In [11]:
n_total = X.shape[0]*X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings/n_total
print(f"Matrix sparsity: {round(sparsity*100,2)}%")

Matrix sparsity: 6.16%


Vemos que hay un 6% de dispersión que no es mucho y eso indica que el resto tienen problema cold-start (no todas las canciones están calificadas y entonces esas canciones se recomiendan menos)

In [16]:
n_ratings_per_song = X.getnnz(axis=0)

print(f"Hay {len(n_ratings_per_song)} canciones con ratings.")

Hay 32768 canciones con ratings.


#### Normalización de datos

Vamos a calcular la media de puntuación de cada canción. Hay que recordar que las calificaciones son 0 o 1, es decir, no le gusta o le gusta al usuario.

In [17]:
#Calcula la media por canción
sum_ratings_per_song = X.sum(axis=0)
mean_rating_per_song = sum_ratings_per_song/n_ratings_per_song

In [18]:
#Crear una nueva matriz con la media de valoración
X_mean_song = np.tile(mean_rating_per_song, (X.shape[0],1))

In [19]:
X_mean_song.shape 
#Vamos a ver el tamaño de la matriz que será usuarios x canciones valoradas

(610, 32768)

In [20]:
#Normalizamos los datos
X_norm = X - csr_matrix(X_mean_song)

Veamos ahora como están los valores nuevos

In [21]:
print("Original X:", X[0].todense())
print("Normalized X:", X_norm[0].todense())

Original X: [[0 0 0 ... 0 0 0]]
Normalized X: [[-0.63636364 -0.47222222 -0.48780488 ... -0.37142857 -0.5
  -0.51515152]]


#### Recomendador filtrado colaborativo Item-Item con k-NN

Ahora vamos a hacer el recomendador como tal que será mediante el algoritmo de k-NN obtiene las k canciones más similares dado un id de una canción y esa similitud estará definida solamente por las puntuaciones más parecidas

In [22]:
from sklearn.neighbors import NearestNeighbors

def find_similar_songs(song_id, X, song_mapper, song_inv_mapper, k, metric='cosine'):
    """
    Encuentra los k vecinos más cercanos para una canción dada.
    
    Argumentos:
        song_id: id de la canción de interés
        X: matriz de utilidad user-item
        k: número de canción similares que queremos recuperar
        métrica: métrica de distancia para los cálculos de kNN
    
    Salida: devuelve una lista de k ID de canciones similares
    """
    X = X.T
    neighbour_ids = []
    
    song_ind = song_mapper[song_id]
    song_vec = X[song_ind]
    if isinstance(song_vec, (np.ndarray)):
        song_vec = song_vec.reshape(1,-1)
    # usamos k+1 porque la salida kNN incluye la canción de ID = song_id
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(song_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(song_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

Probemos con la segunda canción del dataset

In [23]:
similar_songs = find_similar_songs(1, X_norm, song_mapper, song_inv_mapper, k=10)
similar_songs

[2877, 7406, 3055, 25540, 15795, 14544, 8380, 14749, 5325]

A partir de ids no podemos saber que canciones son las recomendadas por lo que vamos a implementar código para que ese id lo transforme a una lista de información de la canción.

Probamos la recomendación con dos tipos distintos de métricas

1. Métrica de similitud por Coseno

In [24]:
song_titles = dict(zip(songs['songId'], songs['track_name']))

song_id = 30000

similar_song = find_similar_songs(song_id, X_norm, song_mapper, song_inv_mapper, metric='cosine', k=10)
song_title = song_titles[song_id] # Saca el título de la canción
song_row = songs.loc[songs['songId'] == song_id] # Obtiene la columna que contiene el id de esa canción
song_artist = song_row['artist_name'].values[0] #Obtiene el nombre del artista
song_genre = song_row['genre'].values[0] #Obtiene el género de la canción
song_year = song_row['year'].values[0] #Obtiene el año de la canción

print(f"porque has escuchado ... {song_title} de {song_artist} ({song_genre}, {song_year}) te recomendamos:")
for i in similar_song:
    row_i = songs.loc[songs['songId'] == i]
    artist_i = row_i['artist_name'].values[0]
    genre_i = row_i['genre'].values[0]
    year_i = row_i['year'].values[0]
    print(f"{song_titles[i]} de {artist_i} ({genre_i}, {year_i})")

porque has escuchado ... Lion Skin - feat. Jonny Craig and Tyler Carter de Hands Like Houses (hardcore, 2012) te recomendamos:
Barstools and Banjos (feat. The Lacs) de Moccasin Creek (country, 2012)
Surrounded By Control de Noisear (grindcore, 2012)
Wayward Girl Blues de Lottie Kimbrough (blues, 2012)
Terror 404 de Perturbator (club, 2012)
Sailing into the Earth de Gorod (death-metal, 2012)
Tengo un Amor de Toby Love (hip-hop, 2012)
TESKNOTA /SHARAZAN/ de JUSTYNA I PIOTR (disco, 2012)
Red Hot de Billy "The Kid" Emerson (blues, 2012)
Keiki de paniyolo (ambient, 2012)


2. Métrica de similitud por Euclidea

In [25]:
song_id = 30000

similar_song = find_similar_songs(song_id, X_norm, song_mapper, song_inv_mapper, metric='euclidean', k=10)
song_title = song_titles[song_id]
song_row = songs.loc[songs['songId'] == song_id]
song_artist = song_row['artist_name'].values[0]
song_genre = song_row['genre'].values[0]
song_year = song_row['year'].values[0]

print(f"porque has escuchado ... {song_title} de {song_artist} ({song_genre}, {song_year}) te recomendamos:")
for i in similar_song:
    row_i = songs.loc[songs['songId'] == i]
    artist_i = row_i['artist_name'].values[0]
    genre_i = row_i['genre'].values[0]
    year_i = row_i['year'].values[0]
    print(f"{song_titles[i]} de {artist_i} ({genre_i}, {year_i})")

porque has escuchado ... Lion Skin - feat. Jonny Craig and Tyler Carter de Hands Like Houses (hardcore, 2012) te recomendamos:
Surrounded By Control de Noisear (grindcore, 2012)
Headfirst de Bulldozer Crash (club, 2012)
Tangomania: I. Infortunio de Cecilio Perera (guitar, 2012)
Traveller de Black Coffee (deep-house, 2012)
Let's Torture Each Other de Forgotten Tomb (black-metal, 2012)
Look Alive de Cut La Roc (breakbeat, 2012)
El Pacino de Bang Data (afrobeat, 2012)
Sailing into the Earth de Gorod (death-metal, 2012)
Heaven Like de Parabelle (hard-rock, 2012)


#### Cold-start

Como se ha comentado antes, cold-star es un problema que aparece cuando no todas las canciones tienen valoraciones o tienen muy pocas. Para solucionarlo crearemos otro recomendador pero basado en contenido. La recomendación será acorde con los atributos que escoja el usuario.

In [26]:
n_songs = songs['songId'].nunique()
print(f"Hay {n_songs} películas en el dataset.")

Hay 50000 películas en el dataset.


Vamos a hacer una función en la que el usuario pase los atributos (posiciones, ya se verá con la interfaz) que quiere incluir para su recomendación. El campo de género se incluirá en otra versión.

In [27]:
#Atributos que usuario puede elegir
col_content = ["popularity", "year",	"danceability",	"energy",	"key",	"loudness",	"mode",	"speechiness",	"acousticness",	"instrumentalness",	"liveness",	"valence",	"tempo",	"duration_ms",	"time_signature"]

In [28]:
#Función para que el usuario elija los atributos que quiera para la recomendación
def choose_content(contents):
    cols = []
    for x in contents:
        cols.append(col_content[x])
    return cols

Para hacer la matriz de correlación es mejor normalizar los valores ya que son muy dispersos entre sí. Esto se sabe porque hay atributos como loudness que va desde -60 a 0 y liveness de 0.0 a 1.0, por lo que hay que hacer sean más o menos parecidos.

In [29]:
# Escalar las características para normalizar las escalas
scaler = StandardScaler()

scaled_features = songs[col_content]
#Para evitar un warning
scaled_features_copy = scaled_features.copy()

scaled_features_copy.loc[:, col_content] = scaler.fit_transform(scaled_features_copy.loc[:, col_content])

scaled_features = scaled_features_copy

Vamos a observar como han cambiado los valores respecto al original

In [30]:
#Columnas normalizadas
scaled_features.head()

,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,3.636637,0.0,-0.280234,-1.388464,-0.358773,-0.314242,0.756294,-0.383319,1.168240,-0.684245,-0.552423,-1.177847,0.373424,-0.119915,-2.000985
1,2.361441,0.0,0.209859,-0.809507,-0.639948,-0.357714,0.756294,-0.513867,0.533630,-0.684207,-0.637493,0.215478,0.604831,-0.288371,0.239057
2,2.857351,0.0,-0.687728,-1.653020,-0.639948,-1.010746,0.756294,-0.464244,0.127129,-0.684107,-0.675677,-1.155613,0.592878,-0.695195,0.239057
3,2.928195,0.0,-0.781341,-1.587840,1.328276,-0.273630,0.756294,-0.433706,1.498705,-0.684245,-0.723045,0.189538,2.817103,0.334373,0.239057
4,2.644818,0.0,-0.572088,0.482602,0.203576,0.570258,-1.322237,-0.480276,-0.649026,-0.630773,-0.576591,-0.888806,1.686805,-0.090487,0.239057


In [31]:
#Columnas originales
songs[col_content].head()

,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,68,2012,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,50,2012,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,57,2012,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,58,2012,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,54,2012,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


Ahora que están normalizados los valores vamos a crear una matriz de similitud del coseno para establecer la relación en los distintos valores y poder usarlos para la recomendación.

In [32]:
cosine_sim_features = cosine_similarity(scaled_features, scaled_features)

Tenemos dos matrices de similitud: una entre géneros (cosine_sim_genre) y otra entre las caracteristicas (cosine_sim_feautures).

En esta función vamos a usar el de las características en esta función.

In [34]:
def content_based_features(title):
    song_idx = dict(zip(songs['track_name'], list(songs.index)))
    n_recommendations = 10

    idx = song_idx[title]
    sim_scores = list(enumerate(cosine_sim_features[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n_recommendations+1)]
    similar_songs = [i[0] for i in sim_scores]

    return similar_songs

Y en esta función usaremos la matriz de los géneros

In [40]:
"""
def content_based_genres(title):
    song_idx = dict(zip(songs['track_name'], list(songs.index)))
    n_recommendations = 10

    idx = song_idx[title]
    sim_scores = list(enumerate(cosine_sim_genre[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n_recommendations+1)]
    similar_songs = [i[0] for i in sim_scores]

    return similar_songs
"""

"\ndef content_based_genres(title):\n    song_idx = dict(zip(songs['track_name'], list(songs.index)))\n    n_recommendations = 10\n\n    idx = song_idx[title]\n    sim_scores = list(enumerate(cosine_sim_genre[idx]))\n    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)\n    sim_scores = sim_scores[1:(n_recommendations+1)]\n    similar_songs = [i[0] for i in sim_scores]\n\n    return similar_songs\n"

Vamos obtener el id de una canción cualquiera para poder hacer el ejemplo.

In [39]:
dict(zip(songs['track_name'], list(songs.index)))['Diamonds']

48926

Mostramos todos los campos para comprobar que se realiza bien la recomendación.

In [37]:
similar_songs = content_based_features('Diamonds')

print(f"Recomendaciones similares a")
print(songs.iloc[48926])
print(f"según su(s) {col_content}:")
songs.iloc[similar_songs]

Recomendaciones similares a
songId                               48926
artist_name                Susanne Sundfør
track_name                        Diamonds
track_id            20UhLm0lDg1WYgYaDXgc0w
popularity                              10
year                                  2012
genre                    singer-songwriter
danceability                         0.347
energy                               0.756
key                                      5
loudness                            -8.702
mode                                     1
speechiness                         0.0653
acousticness                        0.0416
instrumentalness                      0.67
liveness                              0.11
valence                             0.0547
tempo                              119.278
duration_ms                         302733
time_signature                           4
Name: 48926, dtype: object
según su(s) ['popularity', 'year', 'danceability', 'energy', 'key', 'loudness', 'mode

,songId,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
4579,4579,DENIAL OF GOD,Funeral,57BXzbrnndfmgffvMdEZ0n,8,2012,black-metal,0.298,0.868,5,-5.798,1,0.0391,0.000005,0.828,0.0795,0.1060,128.097,403347,4
15702,15702,Robert Hood,The Exodos,5eVyLVOtekxcVzGHAjY8p3,2,2012,detroit-techno,0.342,0.796,8,-8.479,1,0.0447,0.002120,0.805,0.1050,0.0757,118.023,354906,4
43017,43017,Windhand,Heap Wolves - 2023 Remaster,71nDRuUNQlpaPM35cjZdY5,12,2012,psych-rock,0.188,0.782,5,-7.353,1,0.0411,0.000009,0.919,0.0932,0.1480,130.204,289929,4
29931,29931,Ted Nugent,Earthtones,6MOeLUVXWeDQKnQU6y4zlN,8,2012,hard-rock,0.388,0.699,3,-7.948,1,0.0354,0.003340,0.956,0.0861,0.1320,124.028,339787,4
22605,22605,M83,Reunion - The Naked And The Famous Remix,6P26OQ7su1Zkss39JSPXQD,10,2012,french,0.434,0.653,7,-8.506,1,0.0362,0.002870,0.826,0.1030,0.1190,124.045,357875,4
8511,8511,Porcelain Raft,Backwords,3Kw5HHGLUWt801q83fp2Rb,13,2012,chill,0.389,0.683,6,-6.346,1,0.0377,0.126000,0.895,0.0944,0.1190,121.412,247800,4
4409,4409,Watain,Total Funeral,1Nj9BVWk4HSbcUADyke11B,8,2012,black-metal,0.216,0.996,4,-7.105,1,0.1470,0.000007,0.778,0.1100,0.0381,131.644,377213,4
4630,4630,Hypocrisy,Osculum Obscenum,1HLFuQZ5Mjnedn73LYv9SR,6,2012,black-metal,0.248,0.929,2,-8.198,1,0.1250,0.000739,0.774,0.1010,0.0393,137.770,305400,4
4264,4264,Der Weg einer Freiheit,Nachtsam,4WxJVOsxNtHhSc0dYCOIZV,10,2012,black-metal,0.247,0.837,8,-6.493,1,0.0391,0.000024,0.799,0.1210,0.0493,140.030,353250,4
4458,4458,Nachtmystium,Silencing Machine,7iyIS8r86QLdXSLA0p82li,5,2012,black-metal,0.317,0.994,5,-5.717,1,0.0901,0.000002,0.790,0.0326,0.0331,139.948,385493,4
